<h1>About the GTEx Data</h1>

The scripts shown here can be used to pull gene expression data for a user-specified list of genes from the <b>Genotype-Tissue Expression (GTEx)</b> project data. This data set consists of gene expression data from over 50 primary tissue types. Each tissue has samples from multiple individuals and (usually) both sexes, collected from donors across the United States.

The publicly-available data files can be downloaded from https://www.gtexportal.org/home/datasets after creating a free account. Additional raw and protected data is available through dbGaP.

I use the v4 release because it is no longer under embargo as of January 2015.

<h1>Analysis</h1>

The workflow below can be used to calculate average expression levels across males and females separately for each tissue subtype in the data. This was used in <a href="https://www.ncbi.nlm.nih.gov/pubmed/26494842">Slavney et al. 2016 (MBE)</a> to generate the data used in figure 2, but using a different gene list as input.

Unless otherwise noted, all code was written by A. Slavney (2015).

<h1>Input Data</h1>
<h2>Gene expression values (RPKM)</h2>

A matrix of gene expression values in reads per billion. Cols = genes, rows = samples.<br>
File: GTEx_Analysis_V4_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm.gct

<h2>Deidentified subject data</h2>

A file with sex, age, and cause of death data for each tissue sample.<br>
File: GTEx_Data_V4_Annotations_SubjectPhenotypes_DS.txt

<h2>Deidentified sample attribute data</h2>

A file with detailed sample collection and processing information for each tissue sample.<br>
File: GTEx_Data_V4_Annotations_SampleAttributesDS.txt

<h2>Query gene list</h2>

A file containing a list of Ensembl gene and transcript IDs for which we will obtain expression values.<br>
File: Example_genes.txt


In [30]:
## Import modules, define some functions

import numpy, itertools, re, math
import pandas as pd
from collections import defaultdict

def filter_expr(tissue_dict, sample_ids, gene_list):
    ''' Get log2(RPKM) values for all individuals in each tissue '''
    expr = defaultdict(list)
    zeros = []
    for k,v in tissue_dict.items(): # k = tissue, v = sample IDs
        inds = []
        # Make list of column positions in RPKM file for every sample ID of the current sex for this tissue
        for j in v:
            try:
                ind = sample_ids.index(j)
                if sample_ids[ind] != j:
                    print('Sample ID mismatch:',j)
                    raise KeyboardInterrupt
            # Many of these samples are not in the RPKM file
            except ValueError:
                continue
            inds.append(ind)
        # Get expression values for every gene in this tissue
        for g in gene_list:
            # Expr vals of gene g for all individuals in tissue k
            vals = [float(g[n]) for n in inds]
            # Record log2 of non-zero RPKM values for all individual in tissue k
            vals = [math.log(i,2) for i in vals if i>0]
            if vals!=[]:
                # Add entry to expr: k = gene ID, v = [tissue, [expr. vals. from all inds.]]
                expr[g[0].split('.')[0]].append([k,vals])
            else:
                zeros.append(g[0].split('.')[0])
    return([expr,zeros])


def bootstrap(data, n):
    ''' Generate 1000 bootstrap samples of some values and calculate the mean across the samples '''
        
    out = []
    L = len(data)
    # Generate n lists of L random integers, value 0-L, with replacement
    samples = numpy.random.randint(0,L,(n,L))
    for s in samples:
        inds = [data[i] for i in s]
        out.append(numpy.mean(inds))
    return(numpy.mean(out))


def avg_tissue_expr(expr, tissue_list, N, low):
    
    ''' Get the average expression value across individuals for each gene in each tissue,
        after removing expression values below a chosen cutoff value (e.g. 1 RPKM)'''
    
    out_df = pd.DataFrame(columns=[i[1] for i in tissue_list])
    out_df
    
    for k,v in expr.items():
        gene = k
        # Initialize row for this gene with all NA values
        line = ['NA' for t in tissue_list]
        # List of tissues w/ any expr. data for this gene
        g_tissues = [j[0] for j in v]
        if len(g_tissues)==0:
            print('no data:', gene)
        for t in tissue_list: 
            if t[1] in g_tissues:
                ind = g_tissues.index(t[1])
                good_expr = [i for i in v[ind][1] if i>=low]
                if good_expr == []:
                    line[t[0]] = 'NA'
                else:
                    # Add the bootstrapped (across individuals) avg. expr. val. of this gene to the line in the appropriate column (tissue)
                    line[t[0]] = bootstrap(good_expr, N)
        out_df.loc[gene] = line
    return(out_df)
        

<h2>Getting Sex-Specific Data</h2>

If we want to analyze the male and female samples separately, we need to pull the sex information for each sample from the phenotype file.

In [2]:
## For each sex, count the number of individuals with data for at least one tissue type
f_ids, m_ids =[],[]
with open('GTEx_Data_V4_Annotations_SubjectPhenotypes_DS.txt') as PHENS:
    for i in PHENS:
        # Skip comment lines
        if re.match('G+',i):
            i = i.strip('\n').split('\t')
            if i[1] == '2':
                f_ids.append(i[0])
            elif i[1] == '1':
                m_ids.append(i[0])
print(len(f_ids), 'female subjects, ', len(m_ids), 'male subjects')

74 female subjects,  138 male subjects


<h2>Splitting Data by Tissue</h2>

We also want to look at expression within each tissue subtype, so this bit of code sorts the sample IDs in each sex according to tissue.

In [3]:
## For each sex, make a dictionary of sample IDs (each sample = 1 tissue from 1 ind.) for each tissue type
f_tissue_samples = defaultdict(list)
m_tissue_samples = defaultdict(list)
with open('GTEx_Data_V4_Annotations_SampleAttributesDS.txt') as ATTR:
    for i in ATTR:
        # Skip comment lines
        if re.match('G+',i):
            i = i.strip('\n').split('\t')
            # Only interested in Illumina TrueSeq data
            if i[13].strip() == 'TrueSeq.v1':
                samp_id = i[0].strip()
                subj_id = '-'.join(samp_id.split('-')[0:2])
                # Use specific (vs. broad) tissue type
                tissue = i[6]
                # Skip cell line samples
                if re.match('Cells -+',i[6]):
                    continue
                # Add sample IDs to either female or male list
                if subj_id in f_ids:
                    f_tissue_samples[tissue].append(samp_id)
                elif subj_id in m_ids:
                    m_tissue_samples[tissue].append(samp_id)
                else:
                    continue

<h2>Pulling Expression Data for Query Genes</h2>

Next, we will extract expression values for the genes listed in Example_genes.txt from the RPKM file.

In [4]:
## Read in list of genes
q_genes = []
with open('Example_genes.txt') as GENES:
    for i in GENES:
        i = i.strip().split(',')
        q_genes.append(i[0])
        
# Only keep unique gene IDs
q_genes = list(set(q_genes))
print('Query genes:',len(q_genes))
    
## Read in the RPKM file (cols = samples, rows = genes)         
EXPR = open('GTEx_Analysis_2014-01-17_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm.gct')
# Skip comment lines
rpkm = EXPR.read().split('\n')[2:]
EXPR.close()
Sample_ids = rpkm[0].split('\t')

## Retrieve the expression data for query genes
gene_data = []
for i in rpkm:
    i = i.split('\t')
    if i[0].split('.')[0] in q_genes:
        gene_data.append(i)
print('Query genes with expr. data:', len(gene_data))

## Make dictionaries with keys = tissue, values = list of usable expr vals in this tissue for each gene

f_expr = filter_expr(f_tissue_samples, Sample_ids, gene_data)
m_expr = filter_expr(m_tissue_samples, Sample_ids, gene_data)

print('Female, male non-zero genes:', len(f_expr[0]), len(m_expr[0]))


Query genes: 49
Query genes with expr. data: 47
Female, male non-zero genes: 47 47


<h2>Calculate Average Per Gene Per Tissue Expression Values</h2>

Lastly, we'll calculate the bootstrap average log2 expression value of each gene in each tissue for females and males, and store them as pandas dataframes.

In [31]:
""" For each gene, for each tissue, filter and average individual log2(RPKM) values less than the tissue-specific cutoff. """

f_tissue_list = list(enumerate(sorted(f_tissue_samples.keys())))
m_tissue_list = list(enumerate(sorted(m_tissue_samples.keys())))

## Use a minumum value of 0 (corresponding to a log2(1 RPKM))

Female = avg_tissue_expr(f_expr[0], f_tissue_list, 1000, 0)
Male = avg_tissue_expr(m_expr[0], m_tissue_list, 1000, 0)

## Display the Female table
Female

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),Brain - Caudate (basal ganglia),...,Pituitary,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Thyroid,Uterus,Vagina,Whole Blood
ENSG00000134375,3.65648,3.61745,4.31073,3.49193,3.55371,3.22121,3.36649,3.45979,3.81704,3.43346,...,3.65944,3.09673,3.08517,3.44311,3.70013,3.50056,3.62664,3.37235,3.2246,1.4611
ENSG00000176393,4.27664,4.15954,4.6621,3.8653,3.84679,3.39429,4.22995,2.59654,2.5687,2.65992,...,3.80299,4.65408,4.63783,4.72384,5.00541,4.24201,4.83398,4.5147,4.5886,4.28085
ENSG00000163435,2.02319,NA,NA,NA,NA,NA,4.5627,NA,NA,NA,...,1.52047,2.33107,2.14845,3.85308,NA,5.42013,1.3145,1.43154,5.28378,1.19254
ENSG00000170075,0.818561,NA,NA,NA,NA,1.54383,NA,5.16764,4.25626,5.41518,...,NA,0.0219945,NA,NA,NA,NA,NA,NA,0.302602,NA
ENSG00000143862,4.69373,4.63418,4.77154,4.75217,4.70712,4.57441,4.36466,5.12101,5.3832,5.07644,...,4.58491,4.62302,4.58252,4.17222,4.922,4.29906,4.6502,4.77608,4.62941,5.33385
ENSG00000143851,0.511803,0.678116,1.9702,0.542345,0.496112,0.245254,0.693856,NA,NA,2.37271,...,NA,0.0231334,0.468121,1.93301,3.54982,0.948228,0.74489,0.197802,0.533359,3.39633
ENSG00000133067,0.628313,0.48932,NA,5.46013,5.28089,3.25426,1.0898,NA,NA,2.8361,...,2.97291,1.19268,1.13026,NA,2.29211,0.795011,1.94999,1.03992,1.5764,1.84196
ENSG00000077152,0.646395,0.44472,1.9124,0.547754,0.596359,0.59862,0.794656,2.3667,2.6625,2.69583,...,1.14419,1.50964,1.02486,1.8562,2.56812,1.6756,0.55965,1.10764,1.8132,1.14606
ENSG00000077157,3.62044,3.05631,1.55519,6.23894,5.98442,6.4065,5.59927,2.60475,2.89998,2.60772,...,2.08124,2.74533,2.77191,4.54333,3.36492,3.70437,2.89011,5.99601,3.57969,1.39615
ENSG00000143858,NA,NA,0.446559,0.551855,0.260708,0.326138,0.0230013,NA,1.69117,2.48696,...,NA,NA,NA,NA,NA,0.217917,NA,0.505429,0.0463427,NA


Now, we can (for example) compare the male and female expression values for a given gene in a given tissue.

In [45]:
# Female-male log2 fold change for gene ENSG00000143862 in Adrenal Gland:

print(Female.get_value('ENSG00000143862','Adrenal Gland'))
print(Male.get_value('ENSG00000143862','Adrenal Gland'))

logFC = Female.get_value('ENSG00000143862','Adrenal Gland')-Male.get_value('ENSG00000143862','Adrenal Gland')
print(logFC)

# Converted to fold change:
print(2**logFC)

4.771543193905943
4.832115188945298
-0.0605719950393544
0.9588838687912008
